# PyCaret Fugue Integration

[Fugue](https://github.com/fugue-project/fugue) is a low-code unified interface for different computing frameworks such as Spark, Dask and Pandas. PyCaret is using Fugue to support distributed computing scenarios.

## Hello World

### Classification

Let's start with the most standard example, the code is exactly the same as the local version, there is no magic.

In [1]:
from pycaret.datasets import get_data
from pycaret.classification import *

setup(data=get_data("juice"), target = 'Purchase', n_jobs=1)

test_models = models().index.tolist()[:5]

,Description,Value
0,session_id,5901
1,Target,Purchase
2,Target Type,Binary
3,Label Encoded,"CH: 0, MM: 1"
4,Original Data,"(1070, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


`compare_model` is also exactly the same if you don't want to use a distributed system

In [2]:
compare_models(include=test_models, n_select=2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8182,0.8940,0.7256,0.7910,0.7554,0.6112,0.6141,0.1470
nb,Naive Bayes,0.7592,0.8271,0.7567,0.6662,0.7076,0.5047,0.5086,0.0110
dt,Decision Tree Classifier,0.7472,0.7379,0.6596,0.6827,0.6670,0.4641,0.4676,0.0110
knn,K Neighbors Classifier,0.7258,0.7608,0.5898,0.6630,0.6228,0.4089,0.4116,0.0120
svm,SVM - Linear Kernel,0.5560,0.0000,0.7103,0.4297,0.4964,0.1717,0.1762,0.0110


[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=5901, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09)]

Now let's make it distributed, as a toy case, on dask. The only thing changed is an additional parameter `parallel_backend`

In [3]:
from pycaret.parallel import FugueBackend

compare_models(include=test_models, n_select=2, parallel=FugueBackend("dask"))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=5901, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09)]

In order to use Spark as the execution engine, you must have access to a Spark cluster, and you must have a `SparkSession`, let's initialize a local Spark session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Now just change `parallel_backend` to this session object, you make it run on Spark. You must understand this is a toy case. In the real situation, you need to have a SparkSession pointing to a real Spark cluster to enjoy the power of Spark

In [6]:
compare_models(include=test_models, n_select=2, parallel=FugueBackend(spark))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=6084, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09)]

In the end, you can `pull` to get the metrics table

In [7]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8250,0.8916,0.7431,0.8122,0.7737,0.6318,0.6356,0.245
nb,Naive Bayes,0.7741,0.8435,0.7668,0.6766,0.7146,0.5301,0.5375,0.013
dt,Decision Tree Classifier,0.7555,0.7559,0.7034,0.6847,0.6920,0.4898,0.4918,0.020
knn,K Neighbors Classifier,0.7205,0.7745,0.6295,0.6701,0.6424,0.4148,0.4211,0.014
svm,SVM - Linear Kernel,0.4650,0.0000,0.7929,0.3256,0.4573,0.0534,0.0573,0.045


### Regression

It's follows the same pattern as classification.

In [8]:
from pycaret.datasets import get_data
from pycaret.regression import *

setup(data=get_data("insurance"), target = 'charges', n_jobs=1)

test_models = models().index.tolist()[:5]

,Description,Value
0,session_id,8634
1,Target,charges
2,Original Data,"(1338, 7)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(936, 14)"


`compare_model` is also exactly the same if you don't want to use a distributed system

In [9]:
compare_models(include=test_models, n_select=2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,4266.8303,37549910.8000,6084.5220,0.7449,0.6220,0.4313,0.0080
lr,Linear Regression,4255.8081,37555846.8000,6084.1883,0.7447,0.6095,0.4296,0.0110
lasso,Lasso Regression,4255.7184,37554069.8000,6084.1021,0.7447,0.6108,0.4296,0.0110
en,Elastic Net,7392.6075,92238252.8000,9587.9770,0.3892,0.7380,0.9318,0.0100
lar,Least Angle Regression,115867.7365,102148803470.8153,152665.9782,-700.5298,1.4243,19.4205,0.0160


[Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=8634, solver='auto', tol=0.001),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)]

Now let's make it distributed, as a toy case, on dask. The only thing changed is an additional parameter `parallel_backend`

In [3]:
from pycaret.parallel import FugueBackend

compare_models(include=test_models, n_select=2, parallel=FugueBackend("dask"))

[Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
       normalize=False, positive=False, precompute=False, random_state=4030,
       selection='cyclic', tol=0.0001, warm_start=False),
 Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=4030, solver='auto', tol=0.001)]

In order to use Spark as the execution engine, you must have access to a Spark cluster, and you must have a `SparkSession`, let's initialize a local Spark session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Now just change `parallel_backend` to this session object, you make it run on Spark. You must understand this is a toy case. In the real situation, you need to have a SparkSession pointing to a real Spark cluster to enjoy the power of Spark

In [6]:
compare_models(include=test_models, n_select=2, parallel=FugueBackend(spark))

[Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=6877, solver='auto', tol=0.001),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)]

In the end, you can `pull` to get the metrics table

In [7]:
pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,4049.8666,3.384191e+07,5783.6006,0.7636,0.6099,0.4084,0.026
lr,Linear Regression,4281.9666,3.794082e+07,6136.1831,0.7442,0.6194,0.4320,0.015
lasso,Lasso Regression,4132.9517,3.432247e+07,5838.2418,0.7397,0.6279,0.4319,0.022
lar,Least Angle Regression,4270.0485,3.867059e+07,6188.6957,0.7228,0.5928,0.4259,0.038
en,Elastic Net,7226.1141,8.853018e+07,9322.3794,0.3717,0.7420,0.9418,0.013


As you see, the results from the distributed versions can be different from your local versions. In the next section, we will show how to make them identical.

## A more practical case

The above examples are pure toys, to make things work perfectly in a distributed system you must be careful about a few things

### Use a lambda instead of a dataframe in setup

If you directly provide a dataframe in `setup`, this dataset will need to be sent to all worker nodes. If the dataframe is 1G, you have 100 workers, then it is possible your dirver machine will need to send out up to 100G data (depending on specific framework's implementation), then this data transfer becomes a bottleneck itself. Instead, if you provide a lambda function, it doesn't change the local compute scenario, but the driver will only send the function reference to workers, and each worker will be responsible to load the data by themselves, so there is no heavy traffic on the driver side.

### Be deterministic

You should always use `session_id` to make the distributed compute deterministic, otherwise, for the exactly same logic you could get drastically different selection for each run.

### Set n_jobs

It is important to be explicit on n_jobs when you want to run something distributedly, so it will not overuse the local/remote resources. This can also avoid resrouce contention, and make the compute faster.

In [8]:
from pycaret.classification import *

setup(data=lambda: get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=1);

,Description,Value
0,session_id,0
1,Target,Purchase
2,Target Type,Binary
3,Label Encoded,"CH: 0, MM: 1"
4,Original Data,"(1070, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


### Set the appropriate batch_size

`batch_size` parameter helps adjust between load balence and overhead. For each batch, setup will be called only once. So

| Choice |Load Balance|Overhead|Best Scenario|
|---|---|---|---|
|Smaller batch size|Better|Worse|`training time >> data loading time` or `models ~= workers`|
|Larger batch size|Worse|Better|`training time << data loading time` or `models >> workers`|

The default value is set to `1`, meaning we want the best load balance.

### Display progress

In development, you can enable visual effect by `display_remote=True`, but meanwhile you must also enable [Fugue Callback](https://fugue-tutorials.readthedocs.io/tutorials/advanced/rpc.html) so that the driver can monitor worker progress. But it is recommended to turn off display in production.

In [9]:
fconf = {
    "fugue.rpc.server": "fugue.rpc.flask.FlaskRPCServer",  # keep this value
    "fugue.rpc.flask_server.host": "0.0.0.0",  # the driver ip address workers can access
    "fugue.rpc.flask_server.port": "3333",  # the open port on the dirver
    "fugue.rpc.flask_server.timeout": "2 sec",  # the timeout for worker to talk to driver
}

be = FugueBackend("dask", fconf, display_remote=True, batch_size=3, top_only=False)
compare_models(n_select=2, parallel=be)

IntProgress(value=0, description='Processing: ', max=16)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8328,0.8949,0.7585,0.7985,0.7735,0.6416,0.6464,0.018
lr,Logistic Regression,0.8275,0.8964,0.7265,0.8105,0.7589,0.6260,0.6344,0.189
ridge,Ridge Classifier,0.8275,0.0000,0.7479,0.7971,0.7654,0.6299,0.6366,0.020
catboost,CatBoost Classifier,0.8221,0.8967,0.7585,0.7755,0.7624,0.6209,0.6254,1.082
gbc,Gradient Boosting Classifier,0.8195,0.8855,0.7510,0.7760,0.7594,0.6154,0.6193,0.150
rf,Random Forest Classifier,0.8048,0.8792,0.7408,0.7483,0.7397,0.5843,0.5889,0.181
ada,Ada Boost Classifier,0.8021,0.8668,0.7014,0.7639,0.7275,0.5729,0.5776,0.101
lightgbm,Light Gradient Boosting Machine,0.7994,0.8775,0.7299,0.7444,0.7331,0.5730,0.5768,0.055
xgboost,Extreme Gradient Boosting,0.7941,0.8729,0.7228,0.7353,0.7248,0.5609,0.5649,0.283
et,Extra Trees Classifier,0.7820,0.8509,0.7122,0.7214,0.7101,0.5365,0.5428,0.198


[LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                            solver='svd', store_covariance=False, tol=0.0001),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, normalize=False, random_state=0, solver='auto',
                 tol=0.001)]

## Notes

### Spark settings

It is highly recommended to have only 1 worker on each Spark executor, so the worker can fully utilize all cpus (set `spark.task.cpus`). Also when you do this you should explicitly set `n_jobs` in `setup` to the number of cpus of each executor.

```python
executor_cores = 4

spark = SparkSession.builder.config("spark.task.cpus", executor_cores).config("spark.executor.cores", executor_cores).getOrCreate()

setup(data=get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=executor_cores)

compare_models(n_select=2, parallel=FugueBackend(spark))
```

### Databricks

On Databricks, `spark` is the magic variable representing a SparkSession. But there is no difference to use. You do the exactly same thing as before:

```python
compare_models(parallel=FugueBackend(spark))
```

But Databricks, the visualization is difficult, so it may be a good idea to do two things:

* Set `verbose` to False in `setup`
* Set `display_remote` to False in `FugueBackend`

### Dask

Dask has fake distributed modes such as the default (multi-thread) and multi-process modes. The default mode will just work fine (but they are actually running sequentially), and multi-process doesn't work for PyCaret for now because it messes up with PyCaret's global variables. On the other hand, any Spark execution mode will just work fine.

### Local Parallelization

For practical use where you try non-trivial data and models, local parallelization (The eaiest way is to use local Dask as backend as shown above) normally doesn't have performance advantage. Because it's very easy to overload the CPUS on training, increasing the contention of resources. The value of local parallelization is to verify the code and give you confidence that the distributed environment will provide the expected result with much shorter time.

### How to develop 

Distributed systems are powerful but you must follow some good practices to use them:

1. **From small to large:** initially, you must start with a small set of data, for example in `compare_model` limit the models you want to try to a small number of cheap models, and when you verify they work, you can change to a larger model collection.
2. **From local to distributed:** you should follow this sequence: verify small data locally then verify small data distributedly and then verify large data distributedly. The current design makes the transition seamless. You can do these sequentially: `parallel=None` -> `parallel=FugueBackend()` -> `parallel=FugueBackend(spark)`. In the second step, you can replace with a local SparkSession or local dask.